In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,540 kB]
Hit:13

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-15 15:26:40--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.11MB/s    in 0.2s    

2022-09-15 15:26:40 (5.11 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
#url="https://<bucket name>.s3.amazonaws.com/user_payment.csv"
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy-MM-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-08-31 00:00:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [5]:
#Counting the number of rows in the dataframe

df.count()

960872

In [6]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'timestamp')]

In [7]:
 # Drop null values, by default is any which means will drop any rows which has a NA
dropna_df = df.dropna(how='any')
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...|2015-08-31 00:00:00|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...| 

In [8]:
#Checking to see if there were any null rows

dropna_df.count()

960679

In [9]:
review_id_df = dropna_df.select(['review_id','customer_id', 'product_id', 'product_parent', 'review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370|2015-08-31 00:00:00|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722|2015-08-31 00:00:00|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390|2015-08-31 00:00:00|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625|2015-08-31 00:00:00|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221|2015-08-31 00:00:00|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695|2015-08-31 00:00:00|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279|2015-08-31 00:00:00|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113|2015-08-31 00:00:00|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892|2015-08-31 00:00:00|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407|2015-08-31 00:00:00|
|R15C7QEZT0LGZN|    99623

In [10]:
from pyspark.sql.functions import desc
customer_df =dropna_df.select(["customer_id"])\
  .groupby("customer_id")\
  .agg({"customer_id":"count"})
customer_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|    1567510|                 1|
|   19502021|                 1|
|   12819130|                 1|
|   35329257|                 2|
|     108460|                 1|
|    5453476|                 1|
|   29913055|                 1|
|   30717305|                 1|
|    1570030|                 1|
|   19032020|                 1|
|   44178035|                 1|
|   26079415|                 2|
|   14230926|                 1|
|   43478048|                 2|
|   43694941|                 1|
|   12318815|                 3|
|   13731855|                 1|
|     740134|                 1|
|   41956754|                 1|
|   20324070|                 3|
+-----------+------------------+
only showing top 20 rows



In [11]:
customer_df = customer_df.withColumnRenamed("count(customer_id)", "customer_count")
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1567510|             1|
|   19502021|             1|
|   12819130|             1|
|   35329257|             2|
|     108460|             1|
|    5453476|             1|
|   29913055|             1|
|   30717305|             1|
|    1570030|             1|
|   19032020|             1|
|   44178035|             1|
|   26079415|             2|
|   14230926|             1|
|   43478048|             2|
|   43694941|             1|
|   12318815|             3|
|   13731855|             1|
|     740134|             1|
|   41956754|             1|
|   20324070|             3|
+-----------+--------------+
only showing top 20 rows



In [12]:
product_df = dropna_df.select(['product_id', 'product_title']).dropDuplicates(["product_id"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1397458135|New Smart Steampu...|
|1850813491|Casio Men's Watch...|
|1880241196|Bird Watching For...|
|4971850788|Casio Solar Power...|
|4971850813|Newest Casio PRG-...|
|7476764236|Men's stainless s...|
|7506291401|VIP Watch Cell Ph...|
|7560006566|new Stainless Ste...|
|7966770882|CITIZEN Men's Sil...|
|7966770956|Citizen EW0484-59...|
|B00002H4QS|Seiko Dive Watche...|
|B00005OT8V|Timex Men's T7776...|
|B00005OT8Z|Timex Ironman Tri...|
|B00005OT91|Timex Men's T4466...|
|B00005OT93|Timex Dress Metal...|
|B00005OT9R|Xhilaration Lady ...|
|B00005QEME|Men's Timex Inter...|
|B00005RVNI|Kenneth Cole Men'...|
|B0000643Q6|Skagen Women's 10...|
|B0000643Q9|Skagen Men's 170L...|
+----------+--------------------+
only showing top 20 rows



In [13]:
vine_df = dropna_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_df = vine_df.withColumn("star_rating", vine_df["star_rating"].cast("Integer"))
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
|R31U3UH5AZ42LL|          5|            0|          0|   N|
|R2SV659OUJ945Y|          4|            0|          0|   N|
| RA51CP8TR5A2L|          5|            0|          0|   N|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|
| RX27XIIWY5JPB|          4|            0|          0|   N|
|R15C7QEZT0LGZN|          4|            2|          2|   N|
|R361XSS37V0NCZ|          1|            0|          0|   N|
| ROTNLALUAJAUB|          3|            0|          0|   N|
|R2DYX7QU6BGOHR|          5|            

In [14]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [15]:
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"<username>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [16]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [18]:
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [19]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)